# Data Collection

Make sure `PDEControlGym` is correctly installed according to [doc](https://pdecontrolgym.readthedocs.io/en/latest/guide/install.html). 

In [ ]:
import gymnasium as gym
import numpy as np
from pde_control_gym.src import NSReward
from tqdm import tqdm
import gymnasium as gym
import numpy as np
import math
import stable_baselines3
import matplotlib.pyplot as plt
import time 
from tqdm import tqdm
from pde_control_gym.src import NSReward
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3 import PPO
from stable_baselines3 import SAC

from pde_control_gym.src.environments2d.navier_stokes2D import central_difference, laplace
import time 
from tqdm import tqdm
import scipy

### Setup Environment

In [ ]:
# Set initial condition function to be zero
def getInitialCondition(X):
    u = np.zeros_like(X) 
    v = np.zeros_like(X) 
    p = np.zeros_like(X) 
    return u, v, p

# Set up boundary conditions here
boundary_condition = {
    "upper": ["Controllable", "Dirchilet"], 
    "lower": ["Dirchilet", "Dirchilet"], 
    "left": ["Dirchilet", "Dirchilet"], 
    "right": ["Dirchilet", "Dirchilet"], 
}

# Timestep and spatial step for PDE Solver
T = 0.201 # To perform 200 steps
dt = 1e-3
dx, dy = 0.05, 0.05
X, Y = 1, 1
u_target = np.load('target.npz')['u']
v_target = np.load('target.npz')['v']
desire_states = np.stack([u_target, v_target], axis=-1) # (NT, Nx, Ny, 2)
NS2DParameters = {
        "T": T, 
        "dt": dt, 
        "X": X,
        "dx": dx, 
        "Y": Y,
        "dy":dy,
        "action_dim": 1, 
        "reward_class": NSReward(0.1),
        "normalize": False, 
        "reset_init_condition_func": getInitialCondition,
        "boundary_condition": boundary_condition,
        "U_ref": desire_states, 
        "action_ref": 2.0 * np.ones(1000), 
}

# Make the NavierStokes PDE gym
env = gym.make("PDEControlGym-NavierStokes2D", **NS2DParameters)

### Test PPO

In [ ]:
from stable_baselines3 import SAC, PPO
ppoModelPath = "PPO_MODEL_PATH"

model = PPO.load(ppoModelPath)
N_experiments = 50
T = 200
total_reward = 0
for i_id in tqdm(range(N_experiments)):
    obs, _ = env.reset(seed=i_id)
    for t in range(T):
        action, _states = model.predict(obs)
        obs, reward, done, _ , _  = env.step(action)
        total_reward += reward

print(f"Total reward for PPO: {np.round(total_reward/N_experiments, 3)}")

### Test SAC

In [ ]:
sacModelPath = "SAC_MODEL_PATH"
model = SAC.load(sacModelPath)
N_experiments = 50
T = 200
total_reward = 0
for i_id in tqdm(range(N_experiments)):
    obs, _ = env.reset(seed=i_id)
    for t in range(T):
        action, _states = model.predict(obs)
        obs, reward, done, _ , _  = env.step(action)
        total_reward += reward

print(f"Total reward for SAC: {np.round(total_reward/N_experiments,3)}")

### Test Optimization

In [ ]:
from pde_control_gym.src.environments2d.navier_stokes2D import central_difference, laplace
# Model-Based Optimization to optimize action 
def apply_boundary(a1, a2):
    a1[:,[-1, 0]] = 0.
    a1[[-1,0],:] = 0.
    a2[:,[-1, 0]] = 0.
    a2[[-1,0],:] = 0.
    return a1, a2

N_experiments = 50
rewards = []
for i_id in range(N_experiments):
    np.random.seed(i_id)
    total_reward = 0.
    U, V = [], []
    env.reset(seed=0)
    for t in range(T):
        obs, reward, done, _ , _ = env.step(np.random.uniform(2,4)) 
        U.append(env.u)
        V.append(env.v)
        total_reward += reward
    u_target = np.load('target.npz')['u'][1:,:,:]
    v_target = np.load('target.npz')['v'][1:,:,:]
    u_ref = [2 for _ in range(T)]
    for ite in range(1):
        Lam1, Lam2 = [], []
        Lam1.append(np.zeros_like(U[0]))
        Lam2.append(np.zeros_like(U[0]))
        pressure = np.zeros_like(U[0])
        for t in range(T-1):
            lam1, lam2 = Lam1[-1], Lam2[-1]
            dl1dx, dl1dy = central_difference(lam1,"x",dx), central_difference(lam1, "y", dy)
            dl2dx, dl2dy = central_difference(lam2,"x", dx), central_difference(lam2, "y", dy) 
            laplace_l1, laplace_l2 = laplace(lam1, dx, dy), laplace(lam2, dx, dy)
            dlam1dt = - 2 * dl1dx * U[-1-t] - dl1dy * V[-1-t] - dl2dx * V[-1-t] - 0.1 * laplace_l1 + (U[-1-t]-u_target[-1-t])
            dlam2dt = - 2 * dl2dy * V[-1-t] - dl1dy * U[-1-t] - dl2dx * U[-1-t] - 0.1 * laplace_l2 + (V[-1-t]-v_target[-1-t])
            lam1 = lam1 - dt * dlam1dt
            lam2 = lam2 - dt * dlam2dt
            lam1, lam2 = apply_boundary(lam1, lam2)
            pressure = env.solve_pressure(lam1, lam2, pressure)
            lam1 = lam1 - dt * central_difference(pressure, "x", dx)
            lam2 = lam2 - dt * central_difference(pressure, "y", dy)
            lam1, lam2 = apply_boundary(lam1, lam2)
            Lam1.append(lam1)
            Lam2.append(lam2)
        Lam1 = Lam1[::-1]
        actions = []
        for t in range(T):
            dl1dx2 = central_difference(Lam1[t], "y", dy)
            actions.append(u_ref[t] - 0.1/0.1 * sum(dl1dx2[-2, 12:17])*5*dx)
        U, V = [], []
        env.reset(seed=0)
        total_reward = 0.
        for t in tqdm(range(T)):
            obs, reward, done, _ , _ = env.step(actions[t])
            U.append(env.u)
            V.append(env.v)
            total_reward += reward
        print(total_reward)
    rewards.append(total_reward)
print(f"Total reward for Optimization {np.round(np.mean(rewards), 3)}")

In [ ]:
def runSingleEpisodeRL(model, env, parameter):
    terminate = False
    truncate = False

    # Holds the resulting states
    uStorage = []

    # Reset Environment
    obs,__ = env.reset()
    uStorage.append(obs)

    i = 0
    rew = 0
    action_list = []
    while not truncate and not terminate:
        # use backstepping controller
        action = model(obs, parameter)
        obs, rewards, terminate, truncate, info = env.step(action)
        uStorage.append(obs)
        rew += rewards 
    u = np.array(uStorage)
    return rew, u

def RLController(obs, model):
    action, _state = model.predict(obs)
    return action


Relace `PPO_MODEL_PATH` and `SAC_MODEL_PATH` with the pretrained PPO and SAC nominal controller from `PDEControlGym`.

In [ ]:
# Load RL models. 
ppoModelPath = "PPO_MODEL_PATH"
sacModelPath = "SAC_MODEL_PATH"

ppoModel = PPO.load(ppoModelPath)
sacModel = SAC.load(sacModelPath)

def getInitialCondition(X):
    u = np.random.uniform(-0.1, 0.1) * np.ones_like(X) 
    v = 0 * np.ones_like(X) 
    p = 0 * np.ones_like(X) 
    return u, v, p

# Set up boundary conditions here
boundary_condition = {
    "upper": ["Controllable", "Dirchilet"], 
    "lower": ["Dirchilet", "Dirchilet"], 
    "left": ["Dirchilet", "Dirchilet"], 
    "right": ["Dirchilet", "Dirchilet"], 
}

# Timestep and spatial step for PDE Solver
T = 0.201 # To perform 200 steps
dt = 1e-3
dx, dy = 0.05, 0.05
X, Y = 1, 1
u_target = np.load('target.npz')['u']
v_target = np.load('target.npz')['v']
desire_states = np.stack([u_target, v_target], axis=-1) # (NT, Nx, Ny, 2)
NS2DParameters = {
        "T": T, 
        "dt": dt, 
        "X": X,
        "dx": dx, 
        "Y": Y,
        "dy":dy,
        "action_dim": 1, 
        "reward_class": NSReward(0.1),
        "normalize": False, 
        "reset_init_condition_func": getInitialCondition,
        "boundary_condition": boundary_condition,
        "U_ref": desire_states, 
        "action_ref": 2.0 * np.ones(1000), 
}

# Make the NavierStokes PDE gym
env = gym.make("PDEControlGym-NavierStokes2D", **NS2DParameters)

# Model-Based Optimization to optimize action 
def apply_boundary(a1, a2):
    a1[:,[-1, 0]] = 0.
    a1[[-1,0],:] = 0.
    a2[:,[-1, 0]] = 0.
    a2[[-1,0],:] = 0.
    return a1, a2

total_reward = 0.
U, V = [], []
T = 200

rewards = []
times = []
for experiment_i in range(1):
    # np.random.seed(experiment_i)
    # env.reset(seed=400)
    env.reset()
    s = time.time()
    for t in tqdm(range(T)):
        obs, reward, done, _ , _ = env.step(np.random.uniform(2,4)) 
        U.append(env.u)
        V.append(env.v)
        total_reward += reward
    print("Total Reward random:", total_reward)
u_target = np.load('target.npz')['u']
v_target = np.load('target.npz')['v']
u_ref = [2 for _ in range(T)]
xs_opt = []
ys_opt = []
xs_ppo = []
ys_ppo = []
xs_sac = []
ys_sac = []
for ite in tqdm(range(5000)):
    env.reset()
    Lam1, Lam2 = [], []
    Lam1.append(np.zeros_like(U[0]))
    Lam2.append(np.zeros_like(U[0]))
    pressure = np.zeros_like(U[0])
    for t in (range(T-1)):
        lam1, lam2 = Lam1[-1], Lam2[-1]
        dl1dx, dl1dy = central_difference(lam1,"x",dx), central_difference(lam1, "y", dy)
        dl2dx, dl2dy = central_difference(lam2,"x", dx), central_difference(lam2, "y", dy) 
        laplace_l1, laplace_l2 = laplace(lam1, dx, dy), laplace(lam2, dx, dy)
        dlam1dt = - 2 * dl1dx * U[-1-t] - dl1dy * V[-1-t] - dl2dx * V[-1-t] - 0.1 * laplace_l1 + (U[-1-t]-u_target[-1-t])
        dlam2dt = - 2 * dl2dy * V[-1-t] - dl1dy * U[-1-t] - dl2dx * U[-1-t] - 0.1 * laplace_l2 + (V[-1-t]-v_target[-1-t])
        lam1 = lam1 - dt * dlam1dt
        lam2 = lam2 - dt * dlam2dt
        lam1, lam2 = apply_boundary(lam1, lam2)
        pressure = env.solve_pressure(lam1, lam2, pressure)
        lam1 = lam1 - dt * central_difference(pressure, "x", dx)
        lam2 = lam2 - dt * central_difference(pressure, "y", dy)
        lam1, lam2 = apply_boundary(lam1, lam2)
        Lam1.append(lam1)
        Lam2.append(lam2)
    Lam1 = Lam1[::-1]
    actions = []
    for t in (range(T)):
        dl1dx2 = central_difference(Lam1[t], "y", dy)
        actions.append(u_ref[t] - 0.1/0.1 * sum(dl1dx2[-2, :])*5*dx)
    U, V = [], []
    # env.reset(seed=400)
    env.reset()
    total_reward = 0.
    for t in (range(T)):
        obs, reward, done, _ , _ = env.step(actions[t]) # actions === env.U[1:,-1,1,0]
        U.append(env.u)
        V.append(env.v)
        total_reward += reward
        print(env.U[100,-2,10,0])
    xs_opt.append(env.U[:,-1,1,0])
    ys_opt.append(env.U[:,-2,10,0])
    _ppo_r, _u_ppo = runSingleEpisodeRL(RLController, env, ppoModel)
    xs_ppo.append(_u_ppo[:,-1,1,0])
    ys_ppo.append(_u_ppo[:,-2,10,0])
    
    _sac_r, _u_sac = runSingleEpisodeRL(RLController, env, sacModel)
    xs_sac.append(_u_sac[:,-1,1,0])
    ys_sac.append(_u_sac[:,-2,10,0])

    

print(np.stack(xs_opt).shape,np.stack(ys_opt).shape)

print(np.stack(xs_ppo).shape,np.stack(ys_ppo).shape)


print(np.stack(xs_sac).shape,np.stack(ys_sac).shape)
data_opt = {"a": np.stack(xs_opt), "u": np.stack(ys_opt)}
scipy.io.savemat("data_opt_ns.mat", data_opt)

data_ppo = {"a": np.stack(xs_ppo), "u": np.stack(ys_ppo)}
scipy.io.savemat("data_ppo_ns.mat", data_ppo)

data_sac = {"a": np.stack(xs_sac), "u": np.stack(ys_sac)}
scipy.io.savemat("data_sac_ns.mat", data_sac)

# Safety filtering

In [ ]:
import numpy as np
def runSingleEpisodeQP(model, env, parameter):
    terminate = False
    truncate = False

    # Holds the resulting states
    uStorage = []

    # Reset Environment
    obs,__ = env.reset()
    uStorage.append(obs)

    i = 0
    rew = 0
    while not truncate and not terminate:
        # use backstepping controller
        action = model(obs, parameter,i)
        
        obs, rewards, terminate, truncate, info = env.step(action)
        uStorage.append(obs)
        rew += rewards 
        i += 1
    u = np.array(uStorage)
    return rew, u
def QP_filter_Controller(obs, parameter,index):
    return parameter[index+1]

def find_earliest_true(condition):
    # Iterate over the first two dimensions (10 and 8) and check for each slice
    earliest_indices = np.full(condition.shape[:2], 0)  # Initialize with -1 (indicating no valid index)

    for i in range(condition.shape[0]):  # Iterate over first dimension
        for j in range(condition.shape[1]):  # Iterate over second dimension
            # For each slice (i, j), find the earliest index where the condition is True
            # and all subsequent values are also True
            for k in range(condition.shape[2]):
                if not condition[i, j, condition.shape[2]-k-1]: 
                    if k == 0:
                        earliest_indices[i,j] = -1
                    else:
                        earliest_indices[i,j] = condition.shape[2]-k
                    break
    return earliest_indices


Replace the filtered results `FILTER_RESULT_PATH` with the one saved in  `test_cbf_ns.ipynb`.

In [ ]:

RL_1000 = np.load("FILTER_RESULT_PATH")
RL_reward_beforeQP = []
RL_reward_afterQP = []
uBcks_beforeQP_list = []
uBcks_afterQP_list = []
u_target = np.load('target.npz')['u']
v_target = np.load('target.npz')['v']

for i in range(RL_1000["safe_label"].transpose().shape[0]):

    U_list = RL_1000["U_nominal"][:, i]
    Y_list = RL_1000["Y_nominal"][:, i]
    print(i)

    def getInitialConditionFixed(X):
        u = U_list[0] * np.ones_like(X) 
        v = 0 * np.ones_like(X) 
        p = 0 * np.ones_like(X) 
        return u, v, p
    NS2DParametersFixed = NS2DParameters.copy()
    NS2DParametersFixed["reset_init_condition_func"] = getInitialConditionFixed
    envBcksFixed = gym.make("PDEControlGym-NavierStokes2D", **NS2DParametersFixed)
    reward_beforeQP, uBcks_beforeQP = runSingleEpisodeQP(QP_filter_Controller, envBcksFixed, U_list)
    
    uBcks_beforeQP_list.append(uBcks_beforeQP[:,-2,10,0])
    RL_reward_beforeQP.append(reward_beforeQP)

    U_safe_list = RL_1000["U_safe"][:, i]
    def getInitialConditionFixed(X):
        u = U_list[0] * np.ones_like(X) 
        v = 0 * np.ones_like(X) 
        p = 0 * np.ones_like(X) 
        return u, v, p
    NS2DParametersFixed = NS2DParameters.copy()
    NS2DParametersFixed["reset_init_condition_func"] = getInitialConditionFixed
    envBcksFixed = gym.make("PDEControlGym-NavierStokes2D", **NS2DParametersFixed)
    reward_afterQP, uBcks_afterQP = runSingleEpisodeQP(QP_filter_Controller, envBcksFixed, U_safe_list)

    uBcks_afterQP_list.append(uBcks_afterQP[:,-2,10,0])

    RL_reward_afterQP.append(reward_afterQP)
    print(reward_beforeQP,reward_afterQP)


result = np.array([uBcks_beforeQP_list, uBcks_afterQP_list]) 
print(result.shape)

condition = (result[:, :,:]-u_target[:,-2,10] < 0.145) & (result[:, :,:]-u_target[:,-2,10] > -0.145)
earliest_index = find_earliest_true(condition)
valid_earliest_index_beforeQP = earliest_index[0,earliest_index[0,:]>=0]
valid_earliest_index_afterQP = earliest_index[1,earliest_index[1,:]>=0]

print(f"beforeQP PF steps among {valid_earliest_index_beforeQP.shape[0]} PF trajectories", np.mean(result.shape[2] - valid_earliest_index_beforeQP), np.std(result.shape[2] - valid_earliest_index_beforeQP))
print(f"afterQP PF steps among {valid_earliest_index_afterQP.shape[0]} PF trajectories", np.mean(result.shape[2] - valid_earliest_index_afterQP), np.std(result.shape[2] - valid_earliest_index_afterQP))

reward_result = np.array([RL_reward_beforeQP,RL_reward_afterQP])
print("reward: beforeQP and afterQP")
print(np.mean(reward_result, axis=1))
print(np.std(reward_result, axis=1))